In [1]:
import queue as Q
import time
import resource
import sys
import math
import random
import numpy as np
import os


In [2]:
path_to_goal=0
cost_of_path=0
nodes_expanded=0
search_depth=0
max_search_depth=0
running_time=0


#### SKELETON CODE ####

## The Class that Represents the Puzzle

class PuzzleState(object):
    """docstring for PuzzleState"""
    def __init__(self, config, n, parent=None, action="Initial", cost=0):
        if n*n != len(config) or n < 2:
            raise Exception("the length of config is not correct!")
        self.n = n
        self.cost = cost
        self.parent = parent
        self.action = action
        self.dimension = n
        self.config = config
        self.children = {}
        self.blank_index = self.config.index(0)
        for i, item in enumerate(self.config):
            if item == 0:
                self.blank_row = i // self.n
                self.blank_col = i % self.n
                break

    def display(self):
        for i in range(self.n):
            line = []
            offset = i * self.n
            for j in range(self.n):
                line.append(self.config[offset + j])
            print(line)

    def move_left(self):
        if self.blank_col == 0:
            return None
        else:
            target = self.blank_index - 1
            new_config = list(self.config)
            new_config[self.blank_index], new_config[target] = new_config[target], new_config[self.blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Left", cost=self.cost + 1)

    def move_right(self):
        if self.blank_col == self.n - 1:
            return None
        else:
            target = self.blank_index + 1
            new_config = list(self.config)
            new_config[self.blank_index], new_config[target] = new_config[target], new_config[self.blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Right", cost=self.cost + 1)

    def move_up(self):
        if self.blank_row == 0:
            return None
        else:
            target = self.blank_index - self.n
            new_config = list(self.config)
            new_config[self.blank_index], new_config[target] = new_config[target], new_config[self.blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Up", cost=self.cost + 1)

    def move_down(self):
        if self.blank_row == self.n - 1:
            return None
        else:
            target = self.blank_index + self.n
            new_config = list(self.config)
            new_config[self.blank_index], new_config[target] = new_config[target], new_config[self.blank_index]
            return PuzzleState(tuple(new_config), self.n, parent=self, action="Down", cost=self.cost + 1)

    def expand(self):
        """expand the node"""
        # add child nodes in order of UDLR
        if len(self.children) == 0:
            up_child = self.move_up()
            if up_child is not None:
                self.children['Up']=up_child
            down_child = self.move_down()
            if down_child is not None:
                self.children['Down']=down_child
            left_child = self.move_left()
            if left_child is not None:
                self.children['Left']=left_child
            right_child = self.move_right()
            if right_child is not None:
                self.children['Right']=right_child
        return self.children
    
    def goal(self):
        """create the goal configuration"""
        return tuple(list(range(self.n**2)))

In [3]:
### Depth First Search Limited to d = ###

def CreatPuzzle(n):
    '''Generates solvable puzzle'''
    solvable = False
    model = list(range(n**2))
    while solvable is False:
        random.shuffle(model)
        config = tuple(model)
        p = PuzzleState(config,n)
        solvable = Solvable(p)
    return p

### The function that validates puzzles
       
def Solvable(initial_state):
    config = list(initial_state.config)
    config.remove(0)
    o = 0
    for i in range(8):
        for j in range(i+1,8):
            if config[i]>config[j]: o+=1
    if o%2 != 0:
        return False
    else:
        return True

In [24]:
# Function that Writes to output.txt

def writeOutput():

    file = open('output.txt', 'w')
    file.write("path_to_goal: " + str(path_to_goal))
    file.write("\ncost_of_path: " + str(cost_of_path))
    file.write("\nnodes_expanded: " + str(nodes_expanded))
    file.write("\nsearch_depth: " + str(search_depth))
    file.write("\nmax_search_depth: " + str(max_search_depth))
    file.write("\nrunning_time: " + format(running_time, '.8f'))
    file.write("\nmax_ram_usage: " + format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1000.0, '.8f'))    
    file.close()

### Breadth First Search ###

def bfs_search(initial_state):
    """BFS search"""
    global path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth,running_time,remain_in_fontier
    global front_config
    start_time = time.time()
    frontier = Q.deque()
    frontier.append(((),initial_state))
    front_config = set((initial_state.config,))
    explored = set()
    goal = initial_state.goal()
    max_search_depth = 0 
    if initial_state.config == goal: return 'The puzzle is already solved!'
    while frontier:
        sequence,state = frontier.popleft()

        explored.add(state.config)
        children = dict(sorted(state.expand().items()))
#         children = state.expand()
        if len(sequence)+1 > max_search_depth:
            max_search_depth = len(sequence)+1
        for child_action,child_state in children.items():  
            child_config = child_state.config
            if child_config not in explored and child_config not in front_config:
                new_action = sequence + (child_action,)
                if child_config == goal:
                    path_to_goal = list(new_action)
                    cost_of_path = child_state.cost
                    nodes_expanded = len(explored)
                    search_depth = len(new_action)
                    max_search_depth +=1
                    running_time = time.time()-start_time
                    remain_in_fontier = len(frontier)
                    return writeOutput()
                frontier.append(((new_action),child_state))
                front_config.add((child_config))
    return print('Failure')

In [12]:
### Depth First Search Limited to d = ###

def dfs_search(initial_state):
    """DFS search"""
    global path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth,running_time
    
    start_time = time.time()
    frontier = Q.deque()
    frontier.append(((),initial_state))
    front_config = set((initial_state.config,))
    explored = set()
    goal = initial_state.goal()
    if initial_state.config == goal: return 'The puzzle is already solved!'
    
    while frontier:
        sequence,state = frontier.popleft()
        explored.add(state.config)
        try: front_config.remove(state.config) 
        except: pass
#         children = state.expand()
        children = dict(list(state.expand().items())[::-1])
        for child_action,child_state in children.items():
            child_config = child_state.config
            if child_config not in explored and child_config not in front_config:
                new_action = sequence + (child_action,)
                if child_config == goal:
                    path_to_goal = list(new_action)
                    cost_of_path = child_state.cost
                    nodes_expanded = len(explored)
                    search_depth = max_search_depth = len(new_action)
                    running_time = time.time()-start_time
                    return writeOutput()
                frontier.appendleft(((new_action),child_state))
                front_config.add((child_config))
    return 'Failure',len(explored)

In [6]:
### A-star algrithm using Manhattan distance for heuristics ###


def A_star_search(initial_state):

    """A * search"""

    global path_to_goal,cost_of_path,nodes_expanded,search_depth,max_search_depth,running_time,frontier

    start_time = time.time()
    n = initial_state.n
    frontier = Q.deque((((),initial_state,0),))
    explored = set()
    goal = initial_state.goal()
    max_search_depth = 0
    if initial_state.config == goal: return 'The puzzle is already solved!'
    
    while frontier:
        sequence,state,fn = frontier.popleft()
        explored.add(state.config)
        children = state.expand()
        for child_action,child_state in children.items():
            child_config = child_state.config
            if child_config not in explored:
                hn = calculate_manhattan_dist(child_config,n)
                fn = child_state.cost + hn
                
                new_action = sequence + (child_action,)
                if child_config == goal:
                    path_to_goal = list(new_action)
                    cost_of_path = child_state.cost
                    nodes_expanded = len(explored)
                    search_depth = len(new_action)
                    running_time = time.time()-start_time
                    return writeOutput()
                frontier.append(((new_action),child_state,fn))
        frontier = Q.deque(sorted(frontier,key=lambda x:x[2]))

    return 'Failure'



def calculate_total_cost(state):

    """calculate the total estimated cost of a state"""
    return state.cost


def calculate_manhattan_dist(config, n):

    """calculate the manhattan distance of a tile"""
    d = tuple(range(n**2))
    man = sum([abs(config.index(i)%3 - d.index(i)%3) + (abs(config.index(i)//3 - d.index(i)//3)) for i in range(1,n**2)])
    
    return man

def test_goal(puzzle_state,goal):

    """test the state is the goal state or not"""
    
    return puzzle_state.config == goal



In [ ]:
# python3 driver.py dfs 6,1,8,4,0,2,7,3,5

# path_to_goal: ['Up', 'Left', 'Down', ... , 'Up', 'Left', 'Up', 'Left']
# cost_of_path: 46142
# nodes_expanded: 51015
# search_depth: 46142
# max_search_depth: 46142

# python3 driver.py bfs 6,1,8,4,0,2,7,3,5

# path_to_goal: ['Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up']
# cost_of_path: 20
# nodes_expanded: 54094
# search_depth: 20
# max_search_depth: 21

# python3 driver.py ast 6,1,8,4,0,2,7,3,5

# path_to_goal: ['Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up']
# cost_of_path: 20
# nodes_expanded: 696
# search_depth: 20
# max_search_depth: 20

# python3 driver.py dfs 8,6,4,2,1,3,5,7,0

# path_to_goal: ['Up', 'Up', 'Left', ..., , 'Up', 'Up', 'Left']
# cost_of_path: 9612
# nodes_expanded: 9869
# search_depth: 9612
# max_search_depth: 9612


# python3 driver.py bfs 8,6,4,2,1,3,5,7,0

# path_to_goal: ['Left', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Left']
# cost_of_path: 26
# nodes_expanded: 166786
# search_depth: 26
# max_search_depth: 27

# python3 driver.py ast 8,6,4,2,1,3,5,7,0

# path_to_goal: ['Left', 'Up', 'Up', 'Left', 'Down', 'Right', 'Down', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Left']
# cost_of_path: 26
# nodes_expanded: 1585
# search_depth: 26
# max_search_depth: 26

# python3 driver.py dfs 1,2,5,3,4,0,6,7,8

In [25]:
p = PuzzleState((6,1,8,4,0,2,7,3,5),3)

In [28]:
bfs_search(p)

In [16]:
dfs_search(p)

In [26]:
A_star_search(p)

In [29]:
print('''Path to Goal:
Cost_of_path:{}
Nodes_expanded:{}
Search Depth:{}
Max Search Depth:{}
Running Time:{}
'''.format(cost_of_path,nodes_expanded,search_depth,max_search_depth,running_time))

Path to Goal:
Cost_of_path:20
Nodes_expanded:36062
Search Depth:20
Max Search Depth:21
Running Time:0.8714661598205566



In [ ]:
q

In [ ]:
# Main Function that reads in Input and Runs corresponding Algorithm

def main():

    sm = sys.argv[1].lower()

    begin_state = sys.argv[2].split(",")

    begin_state = tuple(map(int, begin_state))

    size = int(math.sqrt(len(begin_state)))

    hard_state = PuzzleState(begin_state, size)

    if sm == "bfs":

        print(bfs_search(hard_state))

    elif sm == "dfs":

        dfs_search(hard_state)

    elif sm == "ast":

        A_star_search(hard_state)

    else:

        print("Enter valid command arguments !")

if __name__ == '__main__':

    main()

In [ ]:
def merge_two_sets(set1, set2): # O(1)? O(n)? O(len(set1) + len(set2))?
      return set1 | set2

In [ ]:
merge_two_sets(set((1,2,3)),set(('a','b','c')))

In [ ]:
set1 = set(('a', 'b', 'c'))
set2 = set((1,2,3))

In [ ]:
set1 | set2

In [ ]:
dict(list(p.expand().items())[::-1])

In [ ]:
f = Q.deque((1,2,3,4))

In [ ]:
f.appendleft()